In [3]:
from RxNorm_API import RxNorm
import pandas as pd
from IPython.display import clear_output

In [7]:
df = pd.read_csv(r'C:\Users\houstonan\Documents\Projects\RxNorm_Mapping\data\raw\pseudo_medications.csv')
df.head()

,ORDER_MNEM_TXT
0,paracetamol
1,midazolam
2,ondansetron
3,sodium chloride 0.9% intravenous solution 500 ...
4,glyceryl trinitrate


In [8]:
medications_df = pd.DataFrame(columns = ['input_term','Name','rxcui','SNOMEDCT','MMSL'])
for value in df['ORDER_MNEM_TXT'][:50]:
    rxnorm = RxNorm()
    term = value

    print('Mapping: ' + term)
    
    rxcui = rxnorm.approximate_term(term = term)
    codes = rxnorm.get_codes(rxcui = rxcui)
    names = rxnorm.get_names(rxcui = rxcui)

    codes['Name'] = names
    codes['rxcui'] = rxcui
    codes['input_term'] = term

    codes = codes[['input_term','Name','rxcui','SNOMEDCT','MMSL']]
    medications_df = medications_df.append(codes, ignore_index=True)

    clear_output()
medications_df.head(30)

,input_term,Name,rxcui,SNOMEDCT,MMSL
0,paracetamol,acetaminophen,161,"387517004, 90332006","BN4992, BN5005, BN52845, GNd00049, IN4119"
1,midazolam,midazolam,6960,"26800000, 373476007","BN130935, GNd00301, IN24970"
2,ondansetron,ondansetron,26225,"108418007, 372487007","GNd00867, IN27290"
3,sodium chloride 0.9% intravenous solution 500 ...,NULL,1807628,NULL,NULL
4,glyceryl trinitrate,nitroglycerin,4917,"387404004, 71759000","BN2324, GNd00321, IN5183"
5,heparin,heparin,5224,"372877000, 84812008",NULL
6,isosorbide dinitrate,isosorbide dinitrate,6058,387332007,"BN2988, BN2996, GNd00268, IN4934"
7,metaraminol,metaraminol,6805,"372728001, 41015006",GNd00702
8,morphine,morphine,7052,"373529000, 73572009",GNd00308
9,compound sodium lactate 1000 mL(s),sodium lactate,56489,"387306001, 55015008","BN1084, GNd00469, IN5487"
